파이썬이 설치되면 디폴트로 사용가능한 모듈 weakref

In [1]:
import sys
import weakref
class C:
    pass
c = C()
c.a = 1
print "refcount - ", sys.getrefcount(c)
print 

d = c
print "refcount - ", sys.getrefcount(c)
print 

r = weakref.ref(c)
print "refcount - ", sys.getrefcount(c)
print 

refcount -  2

refcount -  3

refcount -  3



In [5]:
print r
r.a = 1
print r.a

<weakref at 0000000003C5E728; to 'instance' at 0000000003C57C48>


AttributeError: 'weakref' object has no attribute 'a'

In [7]:
print r
print r()
print c
print r().a
print 

del c
del d
print r()
print r().a

<weakref at 0000000003C5E728; to 'instance' at 0000000003C57C48>
<__main__.C instance at 0x0000000003C57C48>
<__main__.C instance at 0x0000000003C57C48>
1

None


AttributeError: 'NoneType' object has no attribute 'a'

- refcount 가 1이라는 것은 가상머신에서만 reference(참조)하고 있다는 뜻이므로, 가비지 컬렉처가 쓰레기 처리를 함

In [8]:
import sys
import weakref
class C:
    pass

c = C()
c.a = 2
print "refcount -", sys.getrefcount(c) # 객체 c의 레퍼런스 카운트 조회
p = weakref.proxy(c) # 프록시 객체를 만든다
print "refcount -", sys.getrefcount(c) # 객체 c의 레퍼런스 카운트 조회 --> 카운트 불변
print 
print p
print c
print p.a

refcount - 2
refcount - 2

<__main__.C instance at 0x0000000003CB5888>
<__main__.C instance at 0x0000000003CB5888>
2


- weakref에서 ref 보다는 proxy를 더 선호함

In [9]:
import weakref
class C:
    pass

c = C()
r = weakref.ref(c)
p = weakref.proxy(c)
print weakref.getweakrefcount(c)
print weakref.getweakrefs(c)

2
[<weakref at 0000000003CC2228; to 'instance' at 0000000003CB5B08>, <weakproxy at 0000000003CC21D8 to instance at 0000000003CB5B08>]


In [10]:
import weakref
class C:
    pass

c = C()
c.a = 4
d = weakref.WeakValueDictionary() # WeakValueDictionary 객체 생성
print d

d[1] = c # 실제 객체에 대한 약한 참조 아이템 생성
print d.items() # 사전 내용 확인
print d[1].a # 실제 객체의 속성 참조

del c # 실제 객체 삭제
print d.items() # 약한 사전에 해당 객체 아이템도 제거되어 있음

<WeakValueDictionary at 63658760>
[(1, <__main__.C instance at 0x0000000003CB5A88>)]
4
[]


# 반복자

- next 메소드를 갖고 있으면, 반복자!

In [13]:
l = iter([1,2,3])
print l

print l.next()
print l.next()
print l.next()
print dir(l)

1
2
3
['__class__', '__delattr__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__iter__', '__length_hint__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'next']


In [14]:
def f(x):
    print x + 1
    
    
for x in [1,2,3]:
    f(x)

2
3
4


In [15]:
def f(x):
    print x + 1
    
t = iter([1,2,3])
while 1:
    try:
        x = t.next()
    except StopIteration:
        break
    f(x)

2
3
4


In [16]:
class Seq:
    def __init__(self, fname):
        self.file = open(fname)
    #def __getitem__(self, n):
    #    if n == 10:
    #        raise StopIteration
    #    return n
    def __iter__(self):
        return self
    def next(self):
        line = self.file.readline() # 한 라인을 읽는다.
        if not line: 
            raise StopIteration     # 읽을 수 없으면 예외 발생
        return line                 # 읽은 라인을 리턴한다.
    
s = Seq('readme.txt')     # s 인스턴스가 next() 메소드를 지니고 있으므로 s 인스턴스 자체가 반복자임 
for line in s: # 우선 __iter__() 메소드를 호출하여 반복자를 얻고, 반복자에 대해서 for ~ in 구문에 의하여 next() 메소드가 호출됨
    print line,

print 

print Seq('readme.txt')

print list(Seq('readme.txt'))  # list() 내장 함수가 객체를 인수로 받으면 해당 객체의 반복자를 얻어와 next()를 매번 호출하여 각 원소를 얻어온다. 
print tuple(Seq('readme.txt')) # tuple() 내장 함수가 객체를 인수로 받으면 해당 객체의 반복자를 얻어와 next()를 매번 호출하여 각 원소를 얻어온다. 


Hello world
Plase, Add me !

<__main__.Seq instance at 0x0000000003CBCF48>
['Hello world\n', 'Plase, Add me !\n']
('Hello world\n', 'Plase, Add me !\n')


In [19]:
class Seq:
    def __init__(self, fname):
        self.file = open(fname)
    def __getitem__(self, n):
        if n == 10:
            raise StopIteration
        return n

s = Seq('readme.txt')     # s 인스턴스가 next() 메소드를 지니고 있으므로 s 인스턴스 자체가 반복자임 
for line in s: # 우선 __iter__() 메소드를 호출하여 반복자를 얻고, 반복자에 대해서 for ~ in 구문에 의하여 next() 메소드가 호출됨
    print line,

print 

print Seq('readme.txt')

print list(Seq('readme.txt'))  # list() 내장 함수가 객체를 인수로 받으면 해당 객체의 반복자를 얻어와 next()를 매번 호출하여 각 원소를 얻어온다. 
print tuple(Seq('readme.txt')) # tuple() 내장 함수가 객체를 인수로 받으면 해당 객체의 반복자를 얻어와 next()를 매번 호출하여 각 원소를 얻어온다. 


0 1 2 3 4 5 6 7 8 9
<__main__.Seq instance at 0x0000000003CC8188>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


- getitem 보다 next 가 더 우선적으로 반환해준다

In [20]:
f = open('readme.txt')
print "f.next()", f.next()
for line in f:   # f.next() 가 순차적으로 호출됨
    print line,    

f.next() Hello world

Plase, Add me !


# 발생자

In [21]:
def f(a, b):
    c = a * b
    d = a + b
    return c, d

In [23]:
def generate_ints(N):
    for i in range(N):
        yield i

In [33]:
gen = generate_ints(3)
print gen
print gen.next()
print gen.next()
print gen.next()
print gen.next()

<generator object generate_ints at 0x0000000003CB0558>
0
1
2


StopIteration: 

In [32]:
def f(N):
    for i in range(N):
        yield i
a = f(10)
print a.next()
print a.next()
print a.next()
print a.next()
print a.next()
print 

print f(10).next()
print f(10).next()
print f(10).next()

0
1
2
3
4

0
0
0


- f(10).next()가 값이 같은 이유는 f(10)이 모두 다른 객체이기 때문

In [41]:
b = [k for k in range(100) if k % 5 == 0]
print b[0], b[1]
print 

for e in b:
    print e,

0 5

0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95


In [42]:
a = (k for k in range(100) if k % 5 == 0)
print a.next(), a.next()
print 

for e in a:
    print e,

print 
print a[0]

0 5

10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95


TypeError: 'generator' object has no attribute '__getitem__'

- 리스트 내포를 []와 ()의 사용이 따라 다름
    - [] : 리스트
    - () : 발생자

- 발생자(generater)는 next를 사용하여 호출할때만, yield를 사용하여 값을 return하기 때문에, 메모리를 사용하지 않는다!!

In [43]:
print sum((k for k in range(100) if k % 5 == 0))

950


In [44]:
def fibonacci(a = 1, b = 1):
    while 1:
        yield a 
        a, b = b, a + b
        
for k in fibonacci():  # 발생자를 직접 for ~ in 구문에 활용
    if k > 100: 
        break
    print k,

1 1 2 3 5 8 13 21 34 55 89


# 예외 처리

- 에러와 예외는 다르다!

In [45]:
4 + spam*3

NameError: name 'spam' is not defined

In [46]:
a = 10
b = 0
c = a / b

ZeroDivisionError: integer division or modulo by zero

In [47]:
def division():
    for n in range(0, 5):
        print 10.0 / n
division()

ZeroDivisionError: float division by zero

In [48]:
'2' + 2

TypeError: cannot concatenate 'str' and 'int' objects

In [49]:
l = [1, 2]
print l[2]

IndexError: list index out of range

- 알아야할 예외
     - NameError
     - ZeroDivisionError
     - TypeError
     - IndexError
     - KeyError
     - IOError

In [52]:
try:
    print 1.0 / 0.0
except ZeroDivisionError:
    print 'zero division error!!!'
except Exception:
    print 'error!!!'

zero division error!!!


- class C(B)
    - 1) c is b (o)
    - 2) b is c (x)

In [59]:
class B:
    pass
class C(B):
    pass

b = B()
c = C()

print isinstance(c, B)
print isinstance(b, C)

True
False


In [60]:
try:
    spam()
except NameError, msg:
    print 'Error -', msg

Error - name 'spam' is not defined


In [61]:
b = 0.0
name = 'aaa.txt'
try:
    print 1.0 / b
    spam()
    f = open(name, 'r')
    '2' + 2
except NameError:
    print 'NameError !!!'
except ZeroDivisionError:
    print 'ZeroDivisionError !!!'
except (TypeError, IOError):
    print 'TypeError or IOError !!!'
else:
    print 'No Exception !!!'
finally:
    print 'Exit !!!'

ZeroDivisionError !!!
Exit !!!


In [62]:
import os
print os.getcwd()
filename = 't.txt'

try:
    f = open(filename, 'r')
except IOError, msg:
    print msg
else:
    a = float(f.readline())
    try:
        answer = 1.0 / a
    except ZeroDivisionError, msg:
        print msg
    else:
        print answer
    finally:
        print "Finally!!!"
        f.close()

C:\Users\gamer\Desktop\Python
1.0
Finally!!!


In [65]:
class SquareSeq:
    def __init__(self, n):
        self.n = n
    def __getitem__(self, k):
        if k >= self.n or k < 0 :
            raise IndexError('첨자 범위를 벗어남') # 첨자 범위를 벗어나면 IndexError 예외를 발생시킴
        return k * k
    def __len__(self):
        return self.n
    
s = SquareSeq(10)
print s[2], s[4]
for x in s:  # IndexError가 발생하는 시점까지 반복한다
    print x,
print s[20]  # 첨자 범위가 넘었다

 4 16
0 1 4 9 16 25 36 49 64 81

IndexError: 泥⑥옄 踰붿쐞瑜�踰쀬뼱��